In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [3]:
robert_frost = '/content/gdrive/MyDrive/NLP/robert_frost.txt.txt'

In [4]:
import numpy as np
import string

np.random.seed(1234)

In [5]:
# the next step will be to create dictionaries to represent our probabilities.The first dictionary, called Initial, will store the initial state distribution.
# The second dictionary, called First Order, will store the First Order transition probabilities, but only for the second word in each sentence.
# This is unlike the First Order Markov model, where the matrix was counted for all one step transitions.  Second Order, will store the second order transition probabilities.

initial = {} #start of a phrase
first_order = {} #second word only
second_order  = {}

In [6]:
def remove_punctuation(s):
    return s.translate(str.maketrans('','',string.punctuation))

In [7]:
# define a function called addtodict, this function will take in three items a dictionary, a key and a value.The basic idea is the key will represent some 
# starting word or pairs of words in the second order case. Value for this dictionary will be a list storing all the possible next words.
# first step is to simply collect the words from the text.

def add2dict(d, k, v):
  if k not in d:
    d[k] = []
  d[k].append(v)

# [cat, dog, cat, cat, dog, mouse...]

In [8]:
#  next step is to traverse our data set where we make use of the function we just wrote to populate the dictionaries above.we'll start by looping over each line
#  of the Robert Frost text file inside the loop. The first step will be to our strip and lowercase the line will then remove the punctuation and then call the 
#  split function to tokenize the sentence. The next step will be to grab the length of our list of tokens, which we will call big T.next step is to look through each 
#  index from zero up to Big T inside the loop or grab the Ith token, called t.
#  check whether or not we are at the end of a sentence, if we are, then I is equal to Big T minus one. In this case, we would like to add a special second order transition 
#  by essentially creating a fake token this fake token is called end in all uppercase letters.The purpose of this is when we are generating new poems, we know that the line
#  should end if we sample the end of line Otherwise our line would go on forever.


for line in open(robert_frost):
  tokens = remove_punctuation(line.rstrip().lower()).split()

  T = len(tokens)
  for i in range(T):
    t = tokens[i]
    if i == 0:
      # measure the distribution of the first word
      initial[t] = initial.get(t, 0.) + 1
    else:
      t_1 = tokens[i-1]
      if i == T - 1:
        # measure probability of ending the line
        add2dict(second_order, (t_1, t), 'END')
      if i == 1:
        # measure distribution of second word
        # given only first word
        add2dict(first_order, t_1, t)
      else:
        t_2 = tokens[i-2]
        add2dict(second_order, (t_2, t_1), t)

In [9]:
# Now, one thing to keep in mind is that at this point, the initial state distribution is the only dictionary that contains actual counts.
# Therefore, it's the only dictionary we can normalize directly for the other dictionaries, they simply store samples of possible next words. We have yet to turn them to counts.
# So in this step, we're going to normalize the initial state distribution first. We'll start by grabbing the sum of all the values, which gives us the total. Of course, this is 
# just equal to the number of lines we encountered. The next step is to live through each key value pair in the dictionary for each pair We're going to take the count, which I've 
# called c and divide that by the total. As you know, this is our maximum likelihood estimate for the probability of each token.

# normalize the distributions
initial_total = sum(initial.values())
for t,c in initial.items():
  initial[t] = c / initial_total

In [10]:
# convert [cat, cat, cat, dog, dog, dog, dog, mouse, ...]
# into {cat: 0.5, dog: 0.4, mouse: 0.1}

# We simply have lists which store all the possible next words.So the following function called list to predict does two things.The first thing it does is it creates a dictionary of counts.
# The second thing it does is once it has the counts, it then normalizes the counts, which converts them into probabilities. so the input argument to this function is T-S, which is just a 
# list of tokens inside the function we'll start by creating a new empty dictionary called D. next step is to grab the total number of samples, which we'll call n. this is length of ts.
# simply increment the value for the token by one each time we encounter the token, as before we use the get method so that we can say the default value is zero. we will have a dictionary 
# storing the counts. the key is the token and the value is the corresponding counts.simply divide the count by the total, which gives us the probability for each token.

def list2pdict(ts):
  # turn each list of possibilties to dict of probabilities
  d = {}
  n = len(ts)
  for t in ts:
    d[t] = d.get(t, 0) + 1
  for t, c in d.items():
    d[t] = c / n
  return d

In [11]:
# We'll start by looping through the first order dictionary which stores the probability for each second word in a sentence  we previously stored in this dictionary was a list 
# of possible next tokens. replace that list of tokens with a dictionary of probabilities.
for t_1, ts in first_order.items():
  # replace list with dictionary of probabilities
  first_order[t_1] = list2pdict(ts)

# print(first_order)
for k, ts in second_order.items():
  second_order[k] = list2pdict(ts)

print(second_order)

{('two', 'roads'): {'diverged': 1.0}, ('roads', 'diverged'): {'in': 1.0}, ('diverged', 'in'): {'a': 1.0}, ('in', 'a'): {'yellow': 0.07142857142857142, 'wood': 0.07142857142857142, 'window': 0.07142857142857142, 'packing': 0.07142857142857142, 'byroad': 0.07142857142857142, 'family': 0.07142857142857142, 'new': 0.07142857142857142, 'row': 0.07142857142857142, 'time': 0.07142857142857142, 'town': 0.07142857142857142, 'book': 0.07142857142857142, 'smother': 0.07142857142857142, 'glass': 0.14285714285714285}, ('yellow', 'wood'): {'END': 1.0}, ('a', 'yellow'): {'wood': 1.0}, ('and', 'sorry'): {'i': 1.0}, ('sorry', 'i'): {'could': 0.5, 'ever': 0.5}, ('i', 'could'): {'not': 0.2, 'END': 0.2, 'have': 0.2, 'see': 0.3, 'do': 0.1}, ('could', 'not'): {'travel': 0.5, 'say': 0.5}, ('travel', 'both'): {'END': 1.0}, ('not', 'travel'): {'both': 1.0}, ('and', 'be'): {'one': 0.5, 'whole': 0.5}, ('be', 'one'): {'traveler': 1.0}, ('one', 'traveler'): {'long': 1.0}, ('traveler', 'long'): {'i': 1.0}, ('i', 's

In [12]:
# create a function that will sample a word, given a dictionary of probabilities.

def sample_word(d):
  # print "d:", d
  p0 = np.random.random()
  # print "p0:", p0
  cumulative = 0
  for t, p in d.items():
    cumulative += p
    if p0 < cumulative:
      return t
  assert(False) # should never get here

In [13]:
initial

{'two': 0.005571030640668524,
 'and': 0.08983286908077995,
 'to': 0.034818941504178275,
 'then': 0.008356545961002786,
 'because': 0.0006963788300835655,
 'though': 0.004874651810584958,
 'had': 0.002785515320334262,
 'in': 0.0201949860724234,
 'oh': 0.002785515320334262,
 'yet': 0.0020891364902506965,
 'i': 0.08217270194986072,
 'somewhere': 0.0006963788300835655,
 'whose': 0.001392757660167131,
 'his': 0.004874651810584958,
 'he': 0.023676880222841225,
 'my': 0.004874651810584958,
 'between': 0.0020891364902506965,
 'the': 0.057103064066852366,
 'of': 0.0201949860724234,
 'but': 0.035515320334261836,
 'some': 0.003481894150417827,
 'from': 0.006963788300835654,
 'is': 0.003481894150417827,
 'natures': 0.0006963788300835655,
 'her': 0.001392757660167131,
 'so': 0.009052924791086351,
 'nothing': 0.001392757660167131,
 'when': 0.006267409470752089,
 'came': 0.0006963788300835655,
 'one': 0.00766016713091922,
 'proclaimed': 0.0006963788300835655,
 'smoothlaid': 0.0006963788300835655,
 'h

In [14]:
# function will generate four lines of a poem at a time. start by answering a loop that runs four times inside the loop.
# initialize an empty list called sentence, which will store the tokens we generate. first step will be to generate the initial word since our dictionary called initial is already a
# word probability dictionary, we can pass this into the sample word function we'll call the result w0. next step is to append W0 to our sentence list.
# next step is to sample the second word in our sentence. his should make use of our dictionary called First Order. we want this to depend on the first word we generated, 
# so we'll index First Order using w0. This will again give us back a probability dictionary. We can then pass this into our simple word function once again.
# This will give us the second word in the sentence, which we will call W1.appended W1 to our sentence list.
# we'll begin by entering an infinite loop inside a loop, we will use our second order dictionary indexed by w0 and w1. this gives us back a probability dictionary, which we 
# can then pass into the sample word function. call the result w2.next step is to check whether or not W2 is equal to our fake and the token.If it is, we don't want to print this out, 
# but simply ends the loop by calling break. final step in this loop is to update the variables zero and W1 one, which represent the most recent previous two words in our sentence.

In [15]:
def generate():
  for i in range(4):
    sentence = []

    # initial word
    w0 = sample_word(initial)
    sentence.append(w0)

    # second word
    w1 = sample_word(first_order[w0])
    sentence.append(w1)

    # second order transitions till END
    while 1:
      w2 = sample_word(second_order[(w0, w1)])
      if w2 == 'END':
        break
      sentence.append(w2)
      w0 = w1
      w1 = w2

    print(' '.join(sentence))


In [16]:
generate()

i went to bed alone and left me
might just as empty
but it isnt as if and thats not all the money goes so fast
you couldnt call it living for it aint


In [17]:
generate()

one level higher than the earth
she has her speel on every single lizard
with whose vast wheels
its to say


In [18]:
# one advantage of storing our probability arrays as dictionaries is that they take advantage of sparcity. We know that there will be lots of zeros.
# zero probability means something which is impossible, but for dictionaries, we don't have to store any impossible transitions.And thus, there are no zeros 
# since we already know that any value which does not exist in the dictionary has 0 probbility. Thus, storing things this way is possibly more space efficient.